# Laboratorio 4 - Regresión y Clustering

## Paula Hípola Gómez, José Ignacio Navas Sanz y Belén Ortega Pérez

## Entregable 2 - Características de los Inmuebles

In [1]:
%config IPCompleter.greedy=True
%reset
#Importamos las librerías necesarias
#Numpy
import numpy as np
#Pandas
import pandas as pd
#Scikit-Learn
import sklearn as sk
#Matplotlib
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

In [3]:
#Importamos el CSV
dataset = pd.read_csv('casas.csv')
#Guardamos una copia del dataset original
datasetOriginal = dataset
#Mostramos los datos
dataset.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,1,2008,WD,Normal,118000


In [4]:
dataset.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


### La columna Id no aporta absolutamente nada extra, es más no es determinante ni descriptiva del valor de una casa por lo que procedemos a eliminarla. Del mismo modo, la variable objetivo es el precio por lo que vamos a sacarlo del dataset. Así obtendremos las 79 variables que describen las casas.

In [5]:
# Creamos el array con la clase de cada registro.
precioDeVenta = dataset["SalePrice"].values
dataset = dataset.drop(columns=['Id', 'SalePrice'])
dataset.head(10)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
5,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal
6,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
7,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Shed,350,11,2009,WD,Normal
8,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml
9,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,1,2008,WD,Normal


In [6]:
#Miramos cuantos datos duplicados hay
numDuplicados = dataset.duplicated().sum()
print('Hay',numDuplicados,'valores duplicados')

Hay 0 valores duplicados


In [7]:
#Miramos cuantos datos nulos hay
columnasNulas = dataset.isnull().sum()
porcentajes = []
valores = []

for col in columnasNulas:
    porcent = (col/dataset.shape[0])*100
    porcentajes.append(porcent)
    
dataFrameNulos = {'Nulos': columnasNulas, 'Porcentaje': porcentajes}
dataFrameNulos = pd.DataFrame(dataFrameNulos)
dataFrameNulos = dataFrameNulos.loc[dataFrameNulos['Nulos'] > 0]
dataFrameNulos

,Nulos,Porcentaje
LotFrontage,259,17.739726
Alley,1369,93.767123
MasVnrType,8,0.547945
MasVnrArea,8,0.547945
BsmtQual,37,2.534247
BsmtCond,37,2.534247
BsmtExposure,38,2.602740
BsmtFinType1,37,2.534247
BsmtFinType2,38,2.602740
Electrical,1,0.068493


In [8]:
#Pasamos las columnas no numericas, los nulos a un valor numerico propio, 'Desconocido'
columnasNoNumericas = dataset.dtypes[dataset.dtypes == 'object']
columnasNoNumericas.index

#Hemos detectado que hay ciertas columnas que el NA implica NO por lo que pondremos ese valor realmente, 
#no es que sea desconocido, es que no tienen esa propiedad las casas

columnasNAesNo = ['Alley','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','FireplaceQu',
                  'GarageType','GarageFinish','GarageQual','GarageCond','PoolQC','Fence','MiscFeature',]

for col in columnasNAesNo:
    dataset[col] = dataset[col].fillna('No')

In [9]:
dataset

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,No,Reg,Lvl,AllPub,Inside,...,0,0,No,No,No,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,No,Reg,Lvl,AllPub,FR2,...,0,0,No,No,No,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,No,IR1,Lvl,AllPub,Inside,...,0,0,No,No,No,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,No,IR1,Lvl,AllPub,Corner,...,0,0,No,No,No,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,No,IR1,Lvl,AllPub,FR2,...,0,0,No,No,No,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,No,Reg,Lvl,AllPub,Inside,...,0,0,No,No,No,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,No,Reg,Lvl,AllPub,Inside,...,0,0,No,MnPrv,No,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,No,Reg,Lvl,AllPub,Inside,...,0,0,No,GdPrv,Shed,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,No,Reg,Lvl,AllPub,Inside,...,0,0,No,No,No,0,4,2010,WD,Normal


In [10]:
#Actualizamos las columnas nulas
#Miramos cuantos datos nulos hay
columnasNulas = dataset.isnull().sum()
porcentajes = []
valores = []

for col in columnasNulas:
    porcent = (col/dataset.shape[0])*100
    porcentajes.append(porcent)
    
dataFrameNulos = {'Nulos': columnasNulas, 'Porcentaje': porcentajes}
dataFrameNulos = pd.DataFrame(dataFrameNulos)
dataFrameNulos = dataFrameNulos.loc[dataFrameNulos['Nulos'] > 0]
dataFrameNulos

,Nulos,Porcentaje
LotFrontage,259,17.739726
MasVnrType,8,0.547945
MasVnrArea,8,0.547945
Electrical,1,0.068493
GarageYrBlt,81,5.547945


In [11]:
#Pasamos las columnas no numericas, los nulos a un valor numerico propio, 'Desconocido'
columnasNoNumericas = dataset.dtypes[dataset.dtypes == 'object']

for col in columnasNoNumericas.index:
    if(col in dataFrameNulos.index):
        dataset[col] = dataset[col].fillna('Desconocido')

In [12]:
#Actualizamos las columnas nulas
#Miramos cuantos datos nulos hay
columnasNulas = dataset.isnull().sum()
porcentajes = []
valores = []

for col in columnasNulas:
    porcent = (col/dataset.shape[0])*100
    porcentajes.append(porcent)
    
dataFrameNulos = {'Nulos': columnasNulas, 'Porcentaje': porcentajes}
dataFrameNulos = pd.DataFrame(dataFrameNulos)
dataFrameNulos = dataFrameNulos.loc[dataFrameNulos['Nulos'] > 0]
dataFrameNulos

,Nulos,Porcentaje
LotFrontage,259,17.739726
MasVnrArea,8,0.547945
GarageYrBlt,81,5.547945


In [13]:
#Como tenemos pocos datos nulos, vamos a poner el valor medio en cada uno de los valores nulos que tenemos.
#Asi no eliminamos datos y podemos completar el dataset con los valores medios.
for col in dataFrameNulos.index:
    dataset[col] = dataset[col].fillna(round(dataset[col].mean(),2))

#### Estas son las columnas de tipo numérico que contienen campos nulos. En estos valores nulos pondremos el valor medio de la columna. Eliminar por ejemplo más de 200 casas por no tener 'LotFrontage' o más de 80 por carecer del año de fabricación del garaje nos parece que es eliminar datos significativos

In [14]:
#Visualizamos el dataset
dataset.head(10)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.00,8450,Pave,No,Reg,Lvl,AllPub,Inside,...,0,0,No,No,No,0,2,2008,WD,Normal
1,20,RL,80.00,9600,Pave,No,Reg,Lvl,AllPub,FR2,...,0,0,No,No,No,0,5,2007,WD,Normal
2,60,RL,68.00,11250,Pave,No,IR1,Lvl,AllPub,Inside,...,0,0,No,No,No,0,9,2008,WD,Normal
3,70,RL,60.00,9550,Pave,No,IR1,Lvl,AllPub,Corner,...,0,0,No,No,No,0,2,2006,WD,Abnorml
4,60,RL,84.00,14260,Pave,No,IR1,Lvl,AllPub,FR2,...,0,0,No,No,No,0,12,2008,WD,Normal
5,50,RL,85.00,14115,Pave,No,IR1,Lvl,AllPub,Inside,...,0,0,No,MnPrv,Shed,700,10,2009,WD,Normal
6,20,RL,75.00,10084,Pave,No,Reg,Lvl,AllPub,Inside,...,0,0,No,No,No,0,8,2007,WD,Normal
7,60,RL,70.05,10382,Pave,No,IR1,Lvl,AllPub,Corner,...,0,0,No,No,Shed,350,11,2009,WD,Normal
8,50,RM,51.00,6120,Pave,No,Reg,Lvl,AllPub,Inside,...,0,0,No,No,No,0,4,2008,WD,Abnorml
9,190,RL,50.00,7420,Pave,No,Reg,Lvl,AllPub,Corner,...,0,0,No,No,No,0,1,2008,WD,Normal


In [15]:
#Comprobamos nulos
print('El dataset tiene un tamaño de:',dataset.shape)
print('Existen',dataset.isnull().sum().sum(),'campos nulos')

El dataset tiene un tamaño de: (1460, 79)
Existen 0 campos nulos


In [16]:
dataset = pd.get_dummies(dataset)
dataset.head(10)

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.00,8450,7,5,2003,2003,196.0,706,0,...,0,0,0,1,0,0,0,0,1,0
1,20,80.00,9600,6,8,1976,1976,0.0,978,0,...,0,0,0,1,0,0,0,0,1,0
2,60,68.00,11250,7,5,2001,2002,162.0,486,0,...,0,0,0,1,0,0,0,0,1,0
3,70,60.00,9550,7,5,1915,1970,0.0,216,0,...,0,0,0,1,1,0,0,0,0,0
4,60,84.00,14260,8,5,2000,2000,350.0,655,0,...,0,0,0,1,0,0,0,0,1,0
5,50,85.00,14115,5,5,1993,1995,0.0,732,0,...,0,0,0,1,0,0,0,0,1,0
6,20,75.00,10084,8,5,2004,2005,186.0,1369,0,...,0,0,0,1,0,0,0,0,1,0
7,60,70.05,10382,7,6,1973,1973,240.0,859,32,...,0,0,0,1,0,0,0,0,1,0
8,50,51.00,6120,7,5,1931,1950,0.0,0,0,...,0,0,0,1,1,0,0,0,0,0
9,190,50.00,7420,5,6,1939,1950,0.0,851,0,...,0,0,0,1,0,0,0,0,1,0


### Ya hemos categorizado nuestros datos para poder procesar correctamente la regresión entre las diferentes variables que aparecen en el dataset.

### Hemos pasado los valores de tipo object (Strings) a 'Desconocido' y posteriormente hemos aplicado el dummies para categorizar los valores a 0, 1, 2... pero que esos valores sean categóricos en lugar de numéricos. 

### Respecto a las columnas con valores nulos numéricos le hemos aplicado la media a los valores faltantes. Consideramos que no modifica en exceso los valores que interpretaremos más adelante

### 1) Realiza todo el preprocesamiento que sea necesario para adaptar las variables que no sean unívocas del dataset y poder usar el algoritmo adecuado. (1 punto)

### 2) Utiliza varias configuraciones teniendo en cuenta el número de grupos a crear y cambiando la medida de distancia entre individuos. Crea una tabla donde se incluya toda la información y el número de iteraciones necesarias para llegar a la solución que presentas. Se considera la mejor solución a aquella que necesita menos iteraciones (1 punto)

### 3) Con la mejor configuración del apartado anterior, utiliza dos criterios para elegir el lugar inicial del punto central de los grupos. Dibuja cómo se van modificando los grupos y cómo van cambiando sus centroides en cada iteración. Obtén una conclusión acerca de dónde deberían situarse los centroides. (1 punto)